<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/langgraph_stream_persistency_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental" "aiosqlite"

In [ ]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
os.environ["TAVILY_API_KEY"] = userdata.get('tavily_api_key')

# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


In [15]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

# Define a Simple Agent State;
# Agent State is accessible to all parts of the graph.
class AgentState(TypedDict):
  messages: Annotated[list[AnyMessage], operator.add]

In [16]:
tool = TavilySearchResults(max_results=2)
print(tool.name)

tavily_search_results_json


In [17]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [18]:
# Define a simple agent
class Agent:
  def __init__(self, model, checkpointer, tools ,system=""):
    self.system = system
    graph = StateGraph(AgentState)
    graph.add_node("llm",self.call_llm)
    graph.add_node("action", self.take_action)
    graph.add_conditional_edges(
        "llm",
        self.exists_action,
        {True: "action", False: END}
    )
    graph.add_edge("action", "llm")
    graph.set_entry_point("llm")
    self.graph = graph.compile(checkpointer = checkpointer)
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def exists_action(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0

  def call_llm(self, state: AgentState):
    messages = state["messages"]
    if self.system:
      messages = [SystemMessage(content=self.system)] + messages
    message = self.model.invoke(messages)
    next_tools = message.tool_calls if message.tool_calls else ["END"]
    #print(f"Next tools: {next_tools}")
    return {'messages': [message]}

  def take_action(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
      print(f"Calling function: {t}")
      result = self.tools[t['name']].invoke(t['args'])
      print(f"Result from calling function: ", result)
      results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
    print("Back to the model")
    return {'messages': results}

In [23]:
prompt = """You are a smart research assistant. Use the search engine to query results. \
You are allowed to make multiple calls (either together or in sequence).\
Only look up information when you are sure of what you want. \
"""
abot = Agent(llm, memory, [tool], system=prompt)

In [24]:
messages = [HumanMessage(content = "What is the weather in Ann Arbor, MI")]
thread = {"configurable": {"thread_id": "1"}}
i = 1
for event in abot.graph.stream({"messages": messages}, thread):
  print("step-", i)
  for v in event.values():
    print(v)
  i += 1

step- 1
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Ann Arbor, MI"}'}, '__gemini_function_call_thought_signatures__': {'c069493f-33af-4527-9c6d-61525914f43e': 'CuMBAXLI2nyn0bt+IfSY9DcXtc0FaLTwYPivdLxKc9WwsMcA/GQ6QogyJSXZff50aWuIzUA36Du/Xgj4qrCdH72iXbk/L2MGaYeqMXJTGAOHAdMdvKBlA1vmvggSdH6nIwbgicXfSk7ai/saHlcfrjKd7eRsm+c2N1zJnp24QV/CtBN/t5ErlSncBx9BvBvf9lIDYP6kTflgnHNEl8T3wDeFfXEwH44VuLGdBOfJi4HFhYDcN8Pnb6mDyH6N8uSOuu5sk1jcGEFGfDARehMplwKjUniJeKEqVcLehNmbUA565SgdZGA='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bbaab-3987-78c2-8134-c79275219a34-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ann Arbor, MI'}, 'id': 'c069493f-33af-4527-9c6d-61525914f43e', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 126, 'output_t

In [17]:
messages2 = [HumanMessage(content="What is the weather in SF")]
for event in abot.graph.stream({"messages": messages2}, thread):
  for v in event.values():
    print(v)


Next tools: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco, CA'}, 'id': '7f2175dc-107f-4197-9fcb-769c1f4899c1', 'type': 'tool_call'}]
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "current weather in San Francisco, CA"}'}, '__gemini_function_call_thought_signatures__': {'7f2175dc-107f-4197-9fcb-769c1f4899c1': 'CuYBAXLI2nws/9W4ZoTql+6q3C4RsJ2CwgDkVIG/nRnDepO9Knsv4gsOSMrmMUV9OSHd7OxDw0SIoAByUbTMprfOjBW5gxfcxzlnETeGMH8KyP3mWf/dic6pOs/lqHuIi4XUTCWAnSyEIAFylvHh1zLYxU/2qEDXpqiyXZDnVjdbGEl7uWb6IuNazBTZ3vEdyVvQE0g5pZLnjlk5RvRnD39gLqh1b6uhTji4cdNtbtvheXA4/eKHl018uhJ4VsM8sFZwtfng6FsS4jry5QsLwy5DQsMvaLBvXn6MBkNQwEjWhZiaYfWmsZQ='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bba91-0b77-7802-bd8c-b491609685a7-0', tool_calls=[{'name': 'tavily_search_results_json'

In [18]:
messages3 = [HumanMessage(content="Which one is the warmer")]
for event in abot.graph.stream({"messages": messages3}, thread):
  for v in event.values():
    print(v)

Next tools: ['END']
{'messages': [AIMessage(content=[{'type': 'text', 'text': "San Francisco is warmer.\n\nAnn Arbor, MI:\n*   Tonight's low: 33°F\n*   Tomorrow's high: 38°F\n\nSan Francisco, CA:\n*   Today's high: 59°F\n*   Today's low: 43°F", 'extras': {'signature': 'CqkFAXLI2nyZ9SHHq1cWXUf3VUdMeyjXSF+aagG0CR/BdySmvWxDm8mF2a2Wc+Wyy0PkewL/cUKcBFFcXtEv3Ip8t3Dr3pylUgyWRr7oqcOsyj3+PxdpfLaD8HPNeh4/+iBThel1pI1MRTmBmOhkYYIzRQslzg6SrQJ2rxPSV1YHa0GoPId99B2PxCcCM84IXTUEjJDiVHWQFCANxI6LV9ulQht20eqG7stCmMmInHwfGr1zxHK0te12TQ8EeYSUnUlu0jL51gccqFk//3EaHyrafqeHTNvaceCyE5mz5AKp/ePyZEPreANwoKC31JverwKh24NCXkefVWcboHHiEnWr7lz/J22/xp+PLudC9S490qLwMPW05uAtoF4BYPNMzIm5PLezFEBnypPr0GwAAShwnEbwKz/Rp30VmsRkkLymC+4NnRSqTDucrnb7MUxurR+l8L1aOn0DZBDPthltFPWtjVe+/tL26oX26NAhrYXvV+jsFspq+2w+cPlJ39pyjiJL8HBzCAtIpqCQ8ZAiqWaGUUsholEv9zXx4Lm6p2x8smjt0/obpGbE2hzrU48K9Y1q3QmxxRTDM/y0VkZw0mY5OyR481e3KHwKWU+wbLtDy5OGXoh8EGoWPEv+hlKWq2cTmgIuG2jcj1PFJAZbV8+XMLmkTTIMOjryGGgP6XEcbCcaAgOg6wXYt0VsK82eebLExyfUqVe1iF5iPKbHmBzxrd

In [ ]:
# Pending: apply async stream call to print each token